In [1]:
import os.path as osp
from math import sqrt

import torch
import torch.nn.functional as F
from rdkit import Chem

from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn.models import AttentiveFP


class GenAttentiveFeatures(object):
    '''
    AttentiveFP 39 node features generation
    '''
    def __init__(self):
        self.symbols = [
            'B', 'C', 'N', 'O', 'F', 'Si', 'P', 'S', 'Cl', 'As', 'Se', 'Br',
            'Te', 'I', 'At', 'other'
        ]

        self.hybridizations = [
            Chem.rdchem.HybridizationType.SP,
            Chem.rdchem.HybridizationType.SP2,
            Chem.rdchem.HybridizationType.SP3,
            Chem.rdchem.HybridizationType.SP3D,
            Chem.rdchem.HybridizationType.SP3D2,
            'other',
        ]

        self.stereos = [
            Chem.rdchem.BondStereo.STEREONONE,
            Chem.rdchem.BondStereo.STEREOANY,
            Chem.rdchem.BondStereo.STEREOZ,
            Chem.rdchem.BondStereo.STEREOE,
        ]

    def __call__(self, data):
        # Generate AttentiveFP features according to Table 1.
        mol = Chem.MolFromSmiles(data.smiles)

        xs = []
        for atom in mol.GetAtoms():
            symbol = [0.] * len(self.symbols)
            symbol[self.symbols.index(atom.GetSymbol())] = 1.
            degree = [0.] * 6
            degree[atom.GetDegree()] = 1.
            formal_charge = atom.GetFormalCharge()
            radical_electrons = atom.GetNumRadicalElectrons()
            hybridization = [0.] * len(self.hybridizations)
            hybridization[self.hybridizations.index(
                atom.GetHybridization())] = 1.
            aromaticity = 1. if atom.GetIsAromatic() else 0.
            hydrogens = [0.] * 5
            hydrogens[atom.GetTotalNumHs()] = 1.
            chirality = 1. if atom.HasProp('_ChiralityPossible') else 0.
            chirality_type = [0.] * 2
            if atom.HasProp('_CIPCode'):
                chirality_type[['R', 'S'].index(atom.GetProp('_CIPCode'))] = 1.

            x = torch.tensor(symbol + degree + [formal_charge] +
                             [radical_electrons] + hybridization +
                             [aromaticity] + hydrogens + [chirality] +
                             chirality_type)
            xs.append(x)

        data.x = torch.stack(xs, dim=0)

        edge_indices = []
        edge_attrs = []
        for bond in mol.GetBonds():
            edge_indices += [[bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]]
            edge_indices += [[bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()]]

            bond_type = bond.GetBondType()
            single = 1. if bond_type == Chem.rdchem.BondType.SINGLE else 0.
            double = 1. if bond_type == Chem.rdchem.BondType.DOUBLE else 0.
            triple = 1. if bond_type == Chem.rdchem.BondType.TRIPLE else 0.
            aromatic = 1. if bond_type == Chem.rdchem.BondType.AROMATIC else 0.
            conjugation = 1. if bond.GetIsConjugated() else 0.
            ring = 1. if bond.IsInRing() else 0.
            stereo = [0.] * 4
            stereo[self.stereos.index(bond.GetStereo())] = 1.

            edge_attr = torch.tensor(
                [single, double, triple, aromatic, conjugation, ring] + stereo)

            edge_attrs += [edge_attr, edge_attr]

        if len(edge_attrs) == 0:
            data.edge_index = torch.zeros((2, 0), dtype=torch.long)
            data.edge_attr = torch.zeros((0, 10), dtype=torch.float)
        else:
            data.edge_index = torch.tensor(edge_indices).t().contiguous()
            data.edge_attr = torch.stack(edge_attrs, dim=0)

        return data


class GenAtomFeatures(object):
    '''
    Our features: @todo
    '''
    pass

In [2]:
from low_inhibitors import LSSInhibitor

for dataset_name in LSSInhibitor.names.keys():
    
    print(dataset_name + "\n")
    
    path = '../../tmp/data'
    
    # use the attentiveFP node and edge features during the mol-2-graph transoformation
    dataset = LSSInhibitor(path, name=dataset_name, pre_transform=GenAttentiveFeatures()).shuffle()
    #dataset = MoleculeNet(path, name='FreeSolv', pre_transform=GenFeatures()).shuffle()
    
    batch_size = 8
    
    # train, valid, test splitting
    N = len(dataset) // 5
    val_dataset = dataset[:N]
    test_dataset = dataset[N:2 * N]
    train_dataset = dataset[2 * N:]
    
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AttentiveFP(in_channels=39, hidden_channels=200, out_channels=1,
                        edge_dim=10, num_layers=2, num_timesteps=2,
                        dropout=0.2).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=10**-2.5,
                                 weight_decay=10**-5)

    def train():
        total_loss = total_examples = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.edge_attr, data.batch)
            loss = F.mse_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += float(loss) * data.num_graphs
            total_examples += data.num_graphs
        return sqrt(total_loss / total_examples)


    @torch.no_grad()
    def test(loader):
        mse = []
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.edge_attr, data.batch)
            mse.append(F.mse_loss(out, data.y, reduction='none').cpu())
        return float(torch.cat(mse, dim=0).mean().sqrt())

    for epoch in range(1, 20):
        train_rmse = train()
        val_rmse = test(val_loader)
        test_rmse = test(test_loader)
        print(f'Epoch: {epoch:03d}, Loss: {train_rmse:.4f} Val: {val_rmse:.4f} '
              f'Test: {test_rmse:.4f}')

mglur2



Processing...
Done!


Epoch: 001, Loss: 4.8805 Val: 2.6074 Test: 2.0794
Epoch: 002, Loss: 1.5539 Val: 1.5016 Test: 0.9939
Epoch: 003, Loss: 1.6238 Val: 1.4550 Test: 0.8971
Epoch: 004, Loss: 1.3461 Val: 1.4121 Test: 1.5372
Epoch: 005, Loss: 1.6839 Val: 1.5527 Test: 1.2935
Epoch: 006, Loss: 1.0844 Val: 1.8530 Test: 1.1539
Epoch: 007, Loss: 0.6595 Val: 0.6510 Test: 0.8699
Epoch: 008, Loss: 1.1648 Val: 1.0015 Test: 0.5520
Epoch: 009, Loss: 0.8770 Val: 2.1871 Test: 1.3245
Epoch: 010, Loss: 1.2326 Val: 1.4161 Test: 0.8773
Epoch: 011, Loss: 0.8108 Val: 0.6467 Test: 0.6047
Epoch: 012, Loss: 0.9743 Val: 1.2660 Test: 0.6447
Epoch: 013, Loss: 0.6636 Val: 1.1525 Test: 0.9411
Epoch: 014, Loss: 0.7340 Val: 0.8203 Test: 1.0171
Epoch: 015, Loss: 0.7038 Val: 1.0294 Test: 1.1670
Epoch: 016, Loss: 0.8767 Val: 1.2462 Test: 0.8705
Epoch: 017, Loss: 0.8543 Val: 1.0563 Test: 1.0143
Epoch: 018, Loss: 0.8400 Val: 1.2766 Test: 0.6722
Epoch: 019, Loss: 0.9393 Val: 0.6759 Test: 0.7361
usp7



Processing...
Done!


Epoch: 001, Loss: 3.7213 Val: 0.9743 Test: 1.8236
Epoch: 002, Loss: 1.9627 Val: 1.0376 Test: 1.7497
Epoch: 003, Loss: 2.0390 Val: 1.9440 Test: 2.3750
Epoch: 004, Loss: 1.5263 Val: 1.3767 Test: 1.1380
Epoch: 005, Loss: 1.6380 Val: 1.2788 Test: 1.1235
Epoch: 006, Loss: 0.9119 Val: 1.0725 Test: 1.5212
Epoch: 007, Loss: 1.1349 Val: 0.9227 Test: 1.1107
Epoch: 008, Loss: 1.0733 Val: 0.9841 Test: 1.1244
Epoch: 009, Loss: 1.0110 Val: 1.0212 Test: 1.0694
Epoch: 010, Loss: 1.1646 Val: 0.9183 Test: 0.9719
Epoch: 011, Loss: 1.2410 Val: 0.9245 Test: 0.9007
Epoch: 012, Loss: 1.0002 Val: 1.0087 Test: 1.1989
Epoch: 013, Loss: 0.9690 Val: 0.7661 Test: 1.3628
Epoch: 014, Loss: 1.2098 Val: 0.9616 Test: 1.0001
Epoch: 015, Loss: 1.0230 Val: 1.0075 Test: 0.7310
Epoch: 016, Loss: 1.0317 Val: 1.0551 Test: 1.1988
Epoch: 017, Loss: 0.7984 Val: 0.6728 Test: 1.0506
Epoch: 018, Loss: 1.0256 Val: 0.6967 Test: 1.0978
Epoch: 019, Loss: 0.8576 Val: 0.6963 Test: 1.1787
mth1



...

Epoch: 001, Loss: 4.0553 Val: 1.7458 Test: 2.1549
Epoch: 002, Loss: 1.5526 Val: 1.5069 Test: 2.0306
Epoch: 003, Loss: 1.2558 Val: 0.8934 Test: 1.2528
Epoch: 004, Loss: 1.1899 Val: 0.8021 Test: 0.8991
Epoch: 005, Loss: 1.0147 Val: 1.0237 Test: 0.8651
Epoch: 006, Loss: 0.8153 Val: 0.9113 Test: 0.9757
Epoch: 007, Loss: 0.8516 Val: 1.0228 Test: 1.0240
Epoch: 008, Loss: 1.0542 Val: 0.6133 Test: 1.0659
Epoch: 009, Loss: 1.0011 Val: 1.0698 Test: 0.8534
Epoch: 010, Loss: 0.8825 Val: 1.0416 Test: 1.0054
Epoch: 011, Loss: 0.8262 Val: 0.7820 Test: 0.9338
Epoch: 012, Loss: 0.9115 Val: 0.9293 Test: 1.0117
Epoch: 013, Loss: 0.7980 Val: 0.8962 Test: 1.1051
Epoch: 014, Loss: 1.0838 Val: 0.9788 Test: 0.9245
Epoch: 015, Loss: 0.9598 Val: 1.0160 Test: 0.8432
Epoch: 016, Loss: 0.8853 Val: 0.9032 Test: 1.0276
Epoch: 017, Loss: 0.8113 Val: 1.0201 Test: 0.7719
Epoch: 018, Loss: 0.8295 Val: 0.8201 Test: 0.9878
Epoch: 019, Loss: 0.8801 Val: 0.6330 Test: 0.8117
notum



Processing...
Done!


Epoch: 001, Loss: 4.7754 Val: 2.3688 Test: 2.3442
Epoch: 002, Loss: 1.8298 Val: 1.5697 Test: 1.7660
Epoch: 003, Loss: 1.5291 Val: 1.2915 Test: 1.7047
Epoch: 004, Loss: 1.2317 Val: 1.2418 Test: 1.3754
Epoch: 005, Loss: 1.1960 Val: 0.9279 Test: 1.2530
Epoch: 006, Loss: 1.1699 Val: 1.0795 Test: 1.3612
Epoch: 007, Loss: 1.2815 Val: 0.9992 Test: 1.1848
Epoch: 008, Loss: 1.2167 Val: 1.1425 Test: 1.1727
Epoch: 009, Loss: 1.0914 Val: 0.8201 Test: 0.9718
Epoch: 010, Loss: 1.0633 Val: 0.7297 Test: 1.1124
Epoch: 011, Loss: 1.0339 Val: 0.5803 Test: 0.8534
Epoch: 012, Loss: 0.9617 Val: 0.9255 Test: 0.9012
Epoch: 013, Loss: 0.9345 Val: 0.7494 Test: 0.7280
Epoch: 014, Loss: 1.0392 Val: 0.9298 Test: 1.0489
Epoch: 015, Loss: 0.8948 Val: 0.9806 Test: 0.7983
Epoch: 016, Loss: 0.9821 Val: 0.6921 Test: 0.9208
Epoch: 017, Loss: 0.7995 Val: 0.8407 Test: 0.8766
Epoch: 018, Loss: 0.7485 Val: 0.9867 Test: 0.8888
Epoch: 019, Loss: 0.7879 Val: 0.6385 Test: 0.7875
eaat3



Processing...
Done!


Epoch: 001, Loss: 2.9798 Val: 0.7909 Test: 0.6286
Epoch: 002, Loss: 0.9298 Val: 0.9553 Test: 0.6951
Epoch: 003, Loss: 0.8461 Val: 1.0131 Test: 0.8238
Epoch: 004, Loss: 0.8431 Val: 0.7860 Test: 0.7575
Epoch: 005, Loss: 0.7538 Val: 0.8112 Test: 0.7652
Epoch: 006, Loss: 0.7526 Val: 0.8422 Test: 0.6739
Epoch: 007, Loss: 0.8011 Val: 0.7066 Test: 0.5714
Epoch: 008, Loss: 0.6407 Val: 0.7735 Test: 0.6801
Epoch: 009, Loss: 0.6375 Val: 0.7796 Test: 0.5521
Epoch: 010, Loss: 0.7099 Val: 0.6746 Test: 0.6961
Epoch: 011, Loss: 0.7681 Val: 0.7387 Test: 0.7763
Epoch: 012, Loss: 0.6398 Val: 0.6161 Test: 0.4979
Epoch: 013, Loss: 0.6780 Val: 0.6216 Test: 0.7026
Epoch: 014, Loss: 0.6904 Val: 0.6415 Test: 0.5827
Epoch: 015, Loss: 0.6768 Val: 0.6631 Test: 0.5251
Epoch: 016, Loss: 0.5230 Val: 0.6775 Test: 0.5417
Epoch: 017, Loss: 0.5880 Val: 0.4540 Test: 0.6829
Epoch: 018, Loss: 0.6002 Val: 0.7044 Test: 0.6021
Epoch: 019, Loss: 0.5952 Val: 0.4666 Test: 0.5077
